In [1]:
import sys,os
import pickle
import anndata
import pandas as pd
import numpy as np
import scanpy as sc
import diffxpy.api as de
import statsmodels.stats.multitest as multi

In [2]:
adata_wta = sc.read_h5ad(open("/lustre/scratch117/cellgen/team283/Kidney-Nanostring/Kidney_AnnData_3.h5ad", "rb"))

In [3]:
pd.DataFrame([adata_wta.obs['region'].iloc[i] + adata_wta.obs['SegmentLabel'].iloc[i] + adata_wta.obs['disease_status'].iloc[i]
for i in range(len(adata_wta.obs['region']))]).value_counts()

glomerulusGeometric SegmentDKD       83
glomerulusGeometric Segmentnormal    66
tubulenegDKD                         24
tubulePanCKDKD                       24
tubulenegnormal                      17
tubulePanCKnormal                    17
dtype: int64

In [4]:
regionArray = ['glomerulus', 'tubule', 'tubule']
segmentArray = ['Geometric Segment', 'neg', 'PanCK']

for i in range(len(segmentArray)):

    order_Control = np.where([adata_wta.obs['region'].iloc[j] == regionArray[i] 
    and adata_wta.obs['SegmentLabel'].iloc[j] == segmentArray[i]
    and adata_wta.obs['disease_status'].iloc[j] == 'normal'
    for j in range(len(adata_wta.obs['region']))])[0]

    print(len(order_Control))
    
    order_DKD = np.where([adata_wta.obs['region'].iloc[j] == regionArray[i] 
    and adata_wta.obs['SegmentLabel'].iloc[j] == segmentArray[i]
    and adata_wta.obs['disease_status'].iloc[j] == 'DKD'
    for j in range(len(adata_wta.obs['region']))])[0]
    
    print(len(order_DKD))
    
    order = np.concatenate([order_Control, order_DKD])

    # Get cpm for both corrected (isc_cpm) and uncorrected data (cpm):
    total_counts = np.sum(adata_wta.X, axis = 1)
    cpm = adata_wta.layers['X_naive']/total_counts.reshape(np.shape(adata_wta.X)[0],1)*10**6
    isc_cpm = adata_wta.layers['X_corrected_cpu']/total_counts.reshape(np.shape(adata_wta.X)[0],1)*10**6

    # Put data into two AnnData objects:
    adata_cpm = sc.AnnData(X=cpm[order],
                                   obs=pd.DataFrame(data = np.concatenate([np.repeat('Control', len(order_Control)),
                                                                          np.repeat('DKD', len(order_DKD))]),
                                                    columns = ['Background']))

    adata_isc_cpm = sc.AnnData(X=isc_cpm[order],
                                   obs=pd.DataFrame(data = np.concatenate([np.repeat('Control', len(order_Control)),
                                                                          np.repeat('DKD', len(order_DKD))]),
                                                    columns = ['Background']))

#     # Perform differential expression with diffxpy for both cases:

    test_rank_cpm = de.test.rank_test(
        data=adata_cpm,
        grouping="Background",
        gene_names = adata_wta.var_names,
        ).summary()

    test_rank_isc_cpm = de.test.rank_test(
        data=adata_isc_cpm,
        grouping="Background",
        gene_names = adata_wta.var_names,
        ).summary()

    # Add FDR:

    test_rank_cpm['FDR'] = multi.multipletests(test_rank_cpm['pval'], method = 'fdr_bh')[1]
    test_rank_isc_cpm['FDR'] = multi.multipletests(test_rank_isc_cpm['pval'], method = 'fdr_bh')[1]
    
    test_rank_cpm.index = test_rank_cpm['gene']
    test_rank_isc_cpm.index = test_rank_isc_cpm['gene']
    test_rank_cpm = test_rank_cpm.drop('gene', axis = 1)
    test_rank_isc_cpm = test_rank_isc_cpm.drop('gene', axis = 1)
    
    adata_wta.varm['DE-Naive-' + regionArray[i] + '_' + segmentArray[i]] = test_rank_cpm
    adata_wta.varm['DE-CountCorrectCpu-' + regionArray[i] + '_' + segmentArray[i]] = test_rank_isc_cpm

66
83


/nfs/users/nfs_a/aa16/.local/lib/python3.7/site-packages/anndata/_core/anndata.py:119: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


17
24
17
24


Also add cell abundance from cell2location to metadata:

In [7]:
cell_density = pd.read_csv('/lustre/scratch117/cellgen/team283/Kidney-Nanostring/resultsLocationModelWTAMultiExperimentHierarchicalGeneLevel_7experiments_33clusters_231locations_9551genes/q05_cell_density_per_mm2.csv',
            index_col=0)

In [10]:
adata_wta.obs = adata_wta.obs.drop(['Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27'], axis = 1)

In [16]:
cell_density.index = adata_wta.obs.index

In [18]:
adata_wta.obs = pd.concat([adata_wta.obs, cell_density], axis = 1)

In [19]:
adata_wta.write_h5ad("/lustre/scratch117/cellgen/team283/Kidney-Nanostring/Kidney_AnnData_4.h5ad")